In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

Packages installed.


In [2]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:

# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,M2A\n,43.648690,-79.385440
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n,43.650964,-79.353041
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n,43.723265,-79.451211
...,...,...,...,...,...
130,M9Y\n,Not assigned\n,M1Z\n,43.648690,-79.385440
131,M2Z\n,Not assigned\n,M3Z\n,43.648690,-79.385440
132,M4Z\n,Not assigned\n,M5Z\n,43.648690,-79.385440
133,M6Z\n,Not assigned\n,M7Z\n,43.648690,-79.385440


In [8]:
neighbors.shape

(135, 5)

In [9]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of M2A
 are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '5eb08baf7828ae001b653788'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [10]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [13]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

M2A

Parkwoods

Victoria Village

Regent Park / Harbourfront

Lawrence Manor / Lawrence Heights

Queen's Park / Ontario Provincial Government

M9A

Malvern / Rouge

M3B

Parkview Hill / Woodbine Gardens

Garden District / Ryerson

Glencairn

M8B

West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale

Rouge Hill / Port Union / Highland Creek

M3C

Woodbine Heights

St. James Town

Humewood-Cedarvale

M8C

Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood

Guildwood / Morningside / West Hill

M3E

The Beaches

Berczy Park

Caledonia-Fairbanks

M8E

M1G

M3G

Leaside

Central Bay Street

Christie

M8G

M1H

Hillcrest Village

Bathurst Manor / Wilson Heights / Downsview North

Thorncliffe Park

Richmond / Adelaide / King

Dufferin / Dovercourt Village

M8H

M1J

Fairview / Henry Farm / Oriole

Northwood Park / York University

East Toronto

Harbourfront East / Union Station / Toronto Islands

Little Portugal / Trinity

M8J

M1K

Bayview Village

Downsvie

In [15]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(540, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2A\n,43.648690,-79.385440,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M2A\n,43.648690,-79.385440,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M2A\n,43.648690,-79.385440,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M2A\n,43.648690,-79.385440,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Parkwoods\n,43.752935,-79.335641,Glen Manor Ravine,43.676821,-79.293942,Trail


In [16]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,4,4,4,4,4,4
Alderwood / Long Branch\n,4,4,4,4,4,4
Bathurst Manor / Wilson Heights / Downsview North\n,4,4,4,4,4,4
Bayview Village\n,4,4,4,4,4,4
Bedford Park / Lawrence Manor East\n,4,4,4,4,4,4
...,...,...,...,...,...,...
Willowdale\n,8,8,8,8,8,8
Willowdale / Newtonbrook\n,4,4,4,4,4,4
Woodbine Heights\n,4,4,4,4,4,4


In [17]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 4 uniques categories.


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Health Food Store,Neighborhood,Pub
0,1,0,M2A\n,0
1,0,1,M2A\n,0
2,0,0,M2A\n,1
3,0,0,M2A\n,0
4,1,0,Parkwoods\n,0


In [19]:
# examine the new dataframe size.

toronto_onehot.shape

(540, 4)

In [20]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Health Food Store,Pub
0,Agincourt\n,0.25,0.25,0.25
1,Alderwood / Long Branch\n,0.25,0.25,0.25
2,Bathurst Manor / Wilson Heights / Downsview No...,0.25,0.25,0.25
3,Bayview Village\n,0.25,0.25,0.25
4,Bedford Park / Lawrence Manor East\n,0.25,0.25,0.25
...,...,...,...,...
126,Willowdale\n,0.25,0.25,0.25
127,Willowdale / Newtonbrook\n,0.25,0.25,0.25
128,Woodbine Heights\n,0.25,0.25,0.25
129,York Mills / Silver Hills\n,0.25,0.25,0.25


In [21]:
# confirm the new size

toronto_grouped.shape

(131, 4)

In [22]:

# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Alderwood / Long Branch
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Bathurst Manor / Wilson Heights / Downsview North
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Bayview Village
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Bedford Park / Lawrence Manor East
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Berczy Park
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub  0.25


----Birch Cliff / Cliffside West
----
               venue  freq
0              Trail  0.25
1  Health Food Store  0.25
2                Pub

In [23]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt\n,Pub,Health Food Store,Trail
1,Alderwood / Long Branch\n,Pub,Health Food Store,Trail
2,Bathurst Manor / Wilson Heights / Downsview No...,Pub,Health Food Store,Trail
3,Bayview Village\n,Pub,Health Food Store,Trail
4,Bedford Park / Lawrence Manor East\n,Pub,Health Food Store,Trail


In [25]:

# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A\n,Not assigned\n,M2A\n,43.648690,-79.385440,0,Pub,Health Food Store,Trail
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641,0,Pub,Health Food Store,Trail
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890,0,Pub,Health Food Store,Trail
3,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n,43.650964,-79.353041,0,Pub,Health Food Store,Trail
4,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n,43.723265,-79.451211,0,Pub,Health Food Store,Trail


In [27]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned\n,0,Pub,Health Food Store,Trail
1,North York\n,0,Pub,Health Food Store,Trail
2,North York\n,0,Pub,Health Food Store,Trail
3,Downtown Toronto\n,0,Pub,Health Food Store,Trail
4,North York\n,0,Pub,Health Food Store,Trail
...,...,...,...,...,...
130,Not assigned\n,0,Pub,Health Food Store,Trail
131,Not assigned\n,0,Pub,Health Food Store,Trail
132,Not assigned\n,0,Pub,Health Food Store,Trail
133,Not assigned\n,0,Pub,Health Food Store,Trail
